# 6 Deformable models
## Exercise: Segmentation and tracking

In [ ]:
import skimage.io
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate
import scipy.linalg
import skimage.draw

In [12]:
%matplotlib qt 

## Simple snake on pluplus image

In [13]:
def make_circular_snake(N, center, radius):
    """ Initialize circular snake as a 2-by-N array."""
    center = center.reshape([2,1])
    angles = np.arange(N)*2*np.pi/N
    return(center+radius*np.array([np.cos(angles), np.sin(angles)]))


def normalize(n):
    return n/np.sqrt(np.sum(n**2,axis=0))


def snake_normals(snake):
    """ Returns snake normals. Expects snake to be 2-by-N array."""
    ds = normalize(np.roll(snake, 1, axis=1) - snake) # (roll shifts the points) get the displacement ds between 2 consecutive points
    tangent = normalize(np.roll(ds,-1,axis=1) + ds) # set tangent as the mean between 2 consecutive displacement
    normal = tangent[[1,0],:] * np.array([-1,1]).reshape([2,1]) # normal is perpendicular to the mean
    return(normal)


## Provided functions
def distribute_points(snake):
    """ Distributes snake points equidistantly. Expects snake to be 2-by-N array."""
    N = snake.shape[1]
    d = np.sqrt(np.sum((np.roll(snake, -1, axis=1)-snake)**2, axis=0)) # length of line segments
    f = scipy.interpolate.interp1d(np.r_[0, np.cumsum(d)], np.c_[snake, snake[:,0:1]])
    return(f(sum(d)*np.arange(N)/N))


def is_crossing(p1, p2, p3, p4):
    """ Check if the line segments (p1, p2) and (p3, p4) cross."""
    crossing = False
    d21 = p2 - p1
    d43 = p4 - p3
    d31 = p3 - p1
    det = d21[0]*d43[1] - d21[1]*d43[0] # Determinant
    if det != 0.0 and d21[0] != 0.0 and d21[1] != 0.0:
        a = d43[0]/d21[0] - d43[1]/d21[1]
        b = d31[1]/d21[1] - d31[0]/d21[0]
        if a != 0.0:
            u = b/a
            if d21[0] > 0:
                t = (d43[0]*u + d31[0])/d21[0]
            else:
                t = (d43[1]*u + d31[1])/d21[1]
            crossing = 0 < u < 1 and 0 < t < 1         
    return crossing


def is_counterclockwise(snake):
    """ Check if points are ordered counterclockwise."""
    return np.dot(snake[0,1:] - snake[0,:-1],
                  snake[1,1:] + snake[1,:-1]) < 0


def remove_intersections(snake):
    """ Reorder snake points to remove self-intersections.
        Arguments: snake represented by a 2-by-N array.
        Returns: snake.
    """
    pad_snake = np.append(snake, snake[:,0].reshape(2,1), axis=1)
    pad_n = pad_snake.shape[1]
    n = pad_n - 1 
    
    for i in range(pad_n - 3):
        for j in range(i + 2, pad_n - 1):
            pts = pad_snake[:,[i, i + 1, j, j + 1]]
            if is_crossing(pts[:,0], pts[:,1], pts[:,2], pts[:,3]):
                # Reverse vertices of smallest loop
                rb = i + 1 # Reverse begin
                re = j     # Reverse end
                if j - i > n // 2:
                    # Other loop is smallest
                    rb = j + 1
                    re = i + n                    
                while rb < re:
                    ia = rb % n
                    rb = rb + 1                    
                    ib = re % n
                    re = re - 1                    
                    pad_snake[:,[ia, ib]] = pad_snake[:,[ib, ia]]                    
                pad_snake[:,-1] = pad_snake[:,0]                
    snake = pad_snake[:,:-1]
    if is_counterclockwise(snake):
        return snake
    else:
        return np.flip(snake, axis=1)
## end provided function
    
def keep_snake_inside(snake, shape):
    """ Contains snake insite the image."""
    snake[snake<0]=0
    snake[0][snake[0]>shape[0]-1] = shape[0]-1 
    snake[1][snake[1]>shape[1]-1] = shape[1]-1 
    return snake

    
def regularization_matrix(N, alpha, beta):
    """ Matrix for smoothing the snake."""
    d = alpha*np.array([-2, 1, 0, 0]) + beta*np.array([-6, 4, -1, 0])
    D = np.fromfunction(lambda i,j: np.minimum((i-j)%N,(j-i)%N), (N,N), dtype=int)
    A = d[np.minimum(D,len(d)-1)]
    return(scipy.linalg.inv(np.eye(N)-A))


def evolve_snake(snake, I, B, step_size):
    """ Single step of snake evolution."""
    
    # Compute mean intensities inside and outside the snake.
    mask = skimage.draw.polygon2mask(I.shape, snake.T)
    m_in = np.mean(I[mask])
    m_out = np.mean(I[~mask])
    
    # Compute the magnitude of the snake displacement given by Eq. (6.1)
    f = scipy.interpolate.RectBivariateSpline(np.arange(I.shape[0]), np.arange(I.shape[1]), I)
    val = f(snake[0],snake[1], grid=False)
    # val = I[snake[0].astype(int), snake[1].astype(int)] # simpler variant without interpolation
    force = 0.5*(m_in-m_out)*(2*val - (m_in+m_out))
    
    
    snake += step_size*force*snake_normals(snake) # external part
    
    snake = np.dot(snake, B) # internal part, ordering influenced by 2-by-N representation of snake
    
    snake = remove_intersections(snake)
    snake = distribute_points(snake)
    snake = keep_snake_inside(snake, I.shape)
    return snake

In [9]:
filename = 'data/plusplus.png'
I = skimage.io.imread(filename).astype(np.float64)
I = np.mean(I,axis=2)/255 # convert to grayscale

nr_points = 100
nr_iter = 100
step_size = 10
alpha = 0.01
beta = 0.1

center = np.array(I.shape)/2
radius = 0.3*np.mean(I.shape)

snake = make_circular_snake(nr_points, center, radius)
B = regularization_matrix(nr_points, alpha, beta)

fig, ax = plt.subplots()
ax.imshow(I, cmap=plt.cm.gray)
ax.plot(np.r_[snake[1],snake[1,0]],np.r_[snake[0],snake[0,0]],'r-')
ax.set_title('Initialization')

for i in range(nr_iter):
    snake = evolve_snake(snake, I, B, step_size)
    ax.clear()
    ax.imshow(I, cmap=plt.cm.gray)
    ax.plot(np.r_[snake[1],snake[1,0]],np.r_[snake[0],snake[0,0]],'r-')
    ax.set_title(f'iteration {i}')
    plt.pause(0.001)

## Tracking amoeba

In [10]:
import imageio

In [11]:
filename = 'data/crawling_amoeba.mov'
vid = imageio.get_reader(filename)
movie = np.array([im for im in vid.iter_data()], dtype=np.float64)/255 # Transforming intensities to doubles between 0 and 1 is advisable, as it might prevent issues in subsequent processing.
movie = np.mean(movie, axis=3) # convert movie frames to grayscale images

#%% settings
nr_points = 100
step_size = 30
alpha = 0.1
beta = 0.1
center = np.array([120,200])
radius = 40

#%% initialization
snake = make_circular_snake(nr_points, center, radius)
B = regularization_matrix(nr_points, alpha, beta)
frame = movie[0]
fig, ax = plt.subplots()
ax.imshow(frame, cmap='gray')
ax.plot(np.r_[snake[1],snake[1,0]],np.r_[snake[0],snake[0,0]],'b-')

for i in range(50):    
    snake = evolve_snake(snake, frame, B, step_size)    
    ax.clear()
    ax.imshow(frame, cmap='gray')
    ax.plot(np.r_[snake[1],snake[1,0]],np.r_[snake[0],snake[0,0]],'b-')
    ax.set_title(f'initialization, iter {i}')
    plt.pause(0.001)
  
      
#%% tracking
for i in range(0,500):
    frame = movie[i] 
    snake = evolve_snake(snake, frame, B, step_size)    
    ax.clear()
    ax.imshow(frame, cmap='gray')
    ax.plot(np.r_[snake[1],snake[1,0]],np.r_[snake[0],snake[0,0]],'b-')
    ax.set_title(f'tracking, frame {i}')
    plt.pause(0.001)

KeyboardInterrupt: 

## Tracking water bear

In [ ]:
filename = 'data/echiniscus.mp4';
vid = imageio.get_reader(filename)
movie = np.array([im for im in vid.iter_data()], dtype=np.float64)/255 # Transforming intensities to doubles between 0 and 1 is advisable, as it might prevent issues in subsequent processing.
gray = (2*movie[:,:,:,2] - movie[:,:,:,1]- movie[:,:,:,0]+2)/4 #  we want to utilize the fact that foreground is yellow while background is blue.


#%% settings
nr_points = 100
step_size = 30
alpha = 0.1
beta = 0.1
center = np.array([120,200])
radius = 40
start_frame = 74

#%% initialization
snake = make_circular_snake(nr_points, center, radius)
B = regularization_matrix(nr_points, alpha, beta)
g = gray[start_frame] # image we operate on
m = movie[start_frame] # image we display
fig, ax = plt.subplots()
ax.imshow(m, cmap='gray')
ax.plot(np.r_[snake[1],snake[1,0]],np.r_[snake[0],snake[0,0]],'r-')

for i in range(50):    
    snake = evolve_snake(snake, g, B, step_size)    
    ax.clear()
    ax.imshow(m, cmap='gray')
    ax.plot(np.r_[snake[1],snake[1,0]],np.r_[snake[0],snake[0,0]],'r-')
    ax.set_title(f'initialization, iter {i}')
    plt.pause(0.001)
  
      
#%% tracking
for i in range(0,300):
    m = movie[start_frame + i] # 2 evolution steps per frame
    g = gray[start_frame + i]
    snake = evolve_snake(snake, g, B, step_size)    
    ax.clear()
    ax.imshow(m, cmap='gray')
    ax.plot(np.r_[snake[1],snake[1,0]],np.r_[snake[0],snake[0,0]],'r-')
    ax.set_title(f'tracking, frame {i}')
    plt.pause(0.001)